Project 2

Ayush Sinha LNU

121334060

Machine Learning (MSML603)

In [ ]:
import numpy as np
import struct

# Function to load MNIST data
def load_mnist_images(filename):
    with open(filename, 'rb') as f:
        # Unpack file header
        _, num_images, rows, cols = struct.unpack('>IIII', f.read(16))
        # Read the image data
        data = np.fromfile(f, dtype=np.uint8).reshape(num_images, rows, cols)
        return data

def load_mnist_labels(filename):
    with open(filename, 'rb') as f:
        # Unpack file header
        _, num_labels = struct.unpack('>II', f.read(8))
        # Read the label data
        data = np.fromfile(f, dtype=np.uint8)
        return data

# Load data
train_images = load_mnist_images('/content/train-images.idx3-ubyte')
train_labels = load_mnist_labels('/content/train-labels.idx1-ubyte')
test_images = load_mnist_images('/content/t10k-images.idx3-ubyte')
test_labels = load_mnist_labels('/content/t10k-labels.idx1-ubyte')

# Normalize the image data
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

# Flatten labels
train_labels = train_labels.flatten()
test_labels = test_labels.flatten()

# Confirm data shapes
train_images.shape, train_labels.shape, test_images.shape, test_labels.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

The MNIST data has been successfully loaded and preprocessed:-

*   Training Images: 60,000 samples of 28x28 pixels.
*   Training Labels: 60,000 corresponding labels.

*   Testing Images: 10,000 samples of 28x28 pixels.
*   Testing Labels: 10,000 corresponding labels.

The pixel values have been normalized to the range [0,1].





In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

# Convert labels to one-hot encoding
train_labels_categorical = to_categorical(train_labels, num_classes=10)
test_labels_categorical = to_categorical(test_labels, num_classes=10)

# Define the feedforward neural network
ffnn_model = Sequential([
    Flatten(input_shape=(28, 28)),  # Flatten the input
    Dense(128, activation='relu'),  # First hidden layer
    Dropout(0.2),                   # Dropout for regularization
    Dense(64, activation='relu'),   # Second hidden layer
    Dense(10, activation='softmax') # Output layer
])

# Compile the model
ffnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
ffnn_history = ffnn_model.fit(
    train_images, train_labels_categorical,
    validation_data=(test_images, test_labels_categorical),
    epochs=10, batch_size=64, verbose=1
)

# Evaluate the model
ffnn_test_loss, ffnn_test_accuracy = ffnn_model.evaluate(test_images, test_labels_categorical, verbose=0)

ffnn_test_accuracy

/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.8282 - loss: 0.5730 - val_accuracy: 0.9606 - val_loss: 0.1306
Epoch 2/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9543 - loss: 0.1504 - val_accuracy: 0.9694 - val_loss: 0.0991
Epoch 3/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9654 - loss: 0.1121 - val_accuracy: 0.9740 - val_loss: 0.0848
Epoch 4/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9725 - loss: 0.0915 - val_accuracy: 0.9740 - val_loss: 0.0837
Epoch 5/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.9765 - loss: 0.0755 - val_accuracy: 0.9750 - val_loss: 0.0784
Epoch 6/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9798 - loss: 0.0663 - val_accuracy: 0.9766 - val_loss: 0.0759
Epoch 7/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.9805 - loss: 0.0608 - val_accuracy: 0.9754 - val_loss: 0.0785
Epoch 8/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.9823 - loss: 0.0524 - val_accuracy: 0.

0.9789999723434448

The above code implements a Feedforward Neural Networks over 10 epochs for a single run and gives us the accuracy.

Here, val_accuracy is the actual Validation/Testing Accuracy which we are concerned with.

In [ ]:
# Import necessary modules
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

# Reload the necessary libraries and preprocess data as the context was reset
import numpy as np
import struct

# Function to load MNIST images
def load_mnist_images(filename):
    with open(filename, 'rb') as f:
        _, num_images, rows, cols = struct.unpack('>IIII', f.read(16))  # Read header
        data = np.fromfile(f, dtype=np.uint8).reshape(num_images, rows, cols)  # Read image data
        return data

# Function to load MNIST labels
def load_mnist_labels(filename):
    with open(filename, 'rb') as f:
        _, num_labels = struct.unpack('>II', f.read(8))  # Read header
        data = np.fromfile(f, dtype=np.uint8)  # Read label data
        return data

# Paths to the MNIST dataset files
train_images_file = '/content/train-images.idx3-ubyte'
train_labels_file = '/content/train-labels.idx1-ubyte'
test_images_file = '/content/t10k-images.idx3-ubyte'
test_labels_file = '/content/t10k-labels.idx1-ubyte'

# Load training and test images, and normalize pixel values to [0, 1]
train_images = load_mnist_images(train_images_file).astype('float32') / 255.0
test_images = load_mnist_images(test_images_file).astype('float32') / 255.0

# Load training and test labels
train_labels = load_mnist_labels(train_labels_file).flatten()
test_labels = load_mnist_labels(test_labels_file).flatten()

# Reshape images to include a channel dimension (for grayscale images: 1 channel)
train_images = train_images.reshape(-1, 28, 28, 1)
test_images = test_images.reshape(-1, 28, 28, 1)

# Convert labels to one-hot encoding
train_labels_categorical = to_categorical(train_labels, num_classes=10)
test_labels_categorical = to_categorical(test_labels, num_classes=10)

# Define the CNN model architecture
cnn_model = Sequential([
    # First Convolutional Layer
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D(pool_size=(2, 2)),  # Max pooling to reduce spatial dimensions
    Dropout(0.25),  # Dropout for regularization

    # Second Convolutional Layer
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),  # Max pooling to further reduce dimensions
    Dropout(0.25),  # Dropout for regularization

    # Flatten the feature maps for the fully connected layers
    Flatten(),

    # Fully Connected Layer
    Dense(128, activation='relu'),  # Dense layer with ReLU activation
    Dropout(0.5),  # Dropout to prevent overfitting

    # Output Layer
    Dense(10, activation='softmax')  # Softmax for multi-class classification
])

# Compile the model
cnn_model.compile(
    optimizer=Adam(learning_rate=0.001),  # Adam optimizer with default learning rate
    loss='categorical_crossentropy',      # Loss function for multi-class classification
    metrics=['accuracy']                  # Metric to monitor training and validation accuracy
)

# Train the model
cnn_history = cnn_model.fit(
    train_images, train_labels_categorical,        # Training data
    validation_data=(test_images, test_labels_categorical),  # Validation data
    epochs=10,                                     # Number of epochs
    batch_size=64,                                 # Batch size
    verbose=1                                      # Display progress during training
)

# Evaluate the model on the test data
cnn_test_loss, cnn_test_accuracy = cnn_model.evaluate(
    test_images, test_labels_categorical, verbose=0
)

# Print test accuracy
cnn_test_accuracy

Epoch 1/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 60s 61ms/step - accuracy: 0.8147 - loss: 0.5656 - val_accuracy: 0.9820 - val_loss: 0.0562
Epoch 2/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 82s 61ms/step - accuracy: 0.9671 - loss: 0.1135 - val_accuracy: 0.9871 - val_loss: 0.0383
Epoch 3/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 82s 61ms/step - accuracy: 0.9749 - loss: 0.0820 - val_accuracy: 0.9893 - val_loss: 0.0299
Epoch 4/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 82s 61ms/step - accuracy: 0.9796 - loss: 0.0692 - val_accuracy: 0.9894 - val_loss: 0.0292
Epoch 5/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 83s 61ms/step - accuracy: 0.9818 - loss: 0.0586 - val_accuracy: 0.9915 - val_loss: 0.0265
Epoch 6/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 81s 61ms/step - accuracy: 0.9843 - loss: 0.0533 - val_accuracy: 0.9916 - val_loss: 0.0258
Epoch 7/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 82s 61ms/step - accuracy: 0.9856 - loss: 0.0472 - val_accuracy: 0.9928 - val_loss: 0.0231
Epoch 8/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 83s 62ms/step - accuracy: 0.9867 - loss: 0.0426 - 

0.9918000102043152

This code implemnts Convolutional Neural Network over 10 epochs for a single run.

In [ ]:
import numpy as np
import struct
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

# Function to load MNIST data
def load_mnist_images(filename):
    with open(filename, 'rb') as f:
        _, num_images, rows, cols = struct.unpack('>IIII', f.read(16))
        data = np.fromfile(f, dtype=np.uint8).reshape(num_images, rows, cols)
        return data

def load_mnist_labels(filename):
    with open(filename, 'rb') as f:
        _, num_labels = struct.unpack('>II', f.read(8))
        data = np.fromfile(f, dtype=np.uint8)
        return data

# Paths to the MNIST dataset files
train_images_file = '/content/train-images.idx3-ubyte'
train_labels_file = '/content/train-labels.idx1-ubyte'
test_images_file = '/content/t10k-images.idx3-ubyte'
test_labels_file = '/content/t10k-labels.idx1-ubyte'

# Load data
train_images = load_mnist_images(train_images_file).astype('float32') / 255.0
test_images = load_mnist_images(test_images_file).astype('float32') / 255.0
train_labels = load_mnist_labels(train_labels_file).flatten()
test_labels = load_mnist_labels(test_labels_file).flatten()

# Reshape data
train_images = train_images.reshape(-1, 28, 28, 1)
test_images = test_images.reshape(-1, 28, 28, 1)

# Convert labels to one-hot encoding
train_labels_categorical = to_categorical(train_labels, num_classes=10)
test_labels_categorical = to_categorical(test_labels, num_classes=10)

# Train and evaluate the feedforward model for multiple runs
num_runs = 5
test_accuracies = []

for run in range(num_runs):
    # Define the feedforward neural network model
    model = Sequential([
        Flatten(input_shape=(28, 28, 1)),          # Flatten the input
        Dense(256, activation='relu'),            # First hidden layer
        BatchNormalization(),                     # Batch normalization
        Dropout(0.3),                             # Dropout for regularization
        Dense(128, activation='relu'),            # Second hidden layer
        BatchNormalization(),                     # Batch normalization
        Dropout(0.3),                             # Dropout for regularization
        Dense(64, activation='relu'),             # Third hidden layer
        BatchNormalization(),                     # Batch normalization
        Dropout(0.2),                             # Dropout for regularization
        Dense(10, activation='softmax')           # Output layer
    ])

    # Compile the model
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    # Train the model
    model.fit(
        train_images, train_labels_categorical,
        validation_data=(test_images, test_labels_categorical),
        epochs=10, batch_size=64, verbose=1
    )

    # Evaluate the model
    _, test_accuracy = model.evaluate(test_images, test_labels_categorical, verbose=0)
    test_accuracies.append(test_accuracy)

# Calculate the average testing accuracy
average_test_accuracy = np.mean(test_accuracies)

# Print the results
print("Test Accuracies for each run:", test_accuracies)
print("Average Test Accuracy:", average_test_accuracy)

Epoch 1/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - accuracy: 0.8039 - loss: 0.6452 - val_accuracy: 0.9544 - val_loss: 0.1389
Epoch 2/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - accuracy: 0.9382 - loss: 0.2060 - val_accuracy: 0.9669 - val_loss: 0.1071
Epoch 3/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - accuracy: 0.9524 - loss: 0.1537 - val_accuracy: 0.9730 - val_loss: 0.0902
Epoch 4/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.9590 - loss: 0.1349 - val_accuracy: 0.9705 - val_loss: 0.0959
Epoch 5/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - accuracy: 0.9626 - loss: 0.1216 - val_accuracy: 0.9769 - val_loss: 0.0730
Epoch 6/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.9668 - loss: 0.1071 - val_accuracy: 0.9782 - val_loss: 0.0726
Epoch 7/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.9707 - loss: 0.0949 - val_accuracy: 0.9788 - val_loss: 0.0687
Epoch 8/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - accuracy: 0.9714 - loss: 0.0943 - val_accuracy

The code is of Feedforward Neural Network which has been run over for 10 epochs and 5 times to get the average accuracy of the network.

The average test accuracy comes out to be: 98.03%.

In [ ]:
import numpy as np
import struct
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

# Function to load MNIST data
def load_mnist_images(filename):
    with open(filename, 'rb') as f:
        _, num_images, rows, cols = struct.unpack('>IIII', f.read(16))
        data = np.fromfile(f, dtype=np.uint8).reshape(num_images, rows, cols)
        return data

def load_mnist_labels(filename):
    with open(filename, 'rb') as f:
        _, num_labels = struct.unpack('>II', f.read(8))
        data = np.fromfile(f, dtype=np.uint8)
        return data

# Paths to the MNIST dataset files
train_images_file = '/content/train-images.idx3-ubyte'
train_labels_file = '/content/train-labels.idx1-ubyte'
test_images_file = '/content/t10k-images.idx3-ubyte'
test_labels_file = '/content/t10k-labels.idx1-ubyte'

# Load data
train_images = load_mnist_images(train_images_file).astype('float32') / 255.0
test_images = load_mnist_images(test_images_file).astype('float32') / 255.0
train_labels = load_mnist_labels(train_labels_file).flatten()
test_labels = load_mnist_labels(test_labels_file).flatten()

# Reshape data
train_images = train_images.reshape(-1, 28, 28, 1)
test_images = test_images.reshape(-1, 28, 28, 1)

# Convert labels to one-hot encoding
train_labels_categorical = to_categorical(train_labels, num_classes=10)
test_labels_categorical = to_categorical(test_labels, num_classes=10)

# Train and evaluate the CNN model for multiple runs
num_runs = 5
test_accuracies = []

for run in range(num_runs):
    # Define the CNN model
    model = Sequential([
        Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        Conv2D(64, kernel_size=(3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])

    # Compile the model
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    # Train the model
    model.fit(
        train_images, train_labels_categorical,
        validation_data=(test_images, test_labels_categorical),
        epochs=10, batch_size=64, verbose=1
    )

    # Evaluate the model
    _, test_accuracy = model.evaluate(test_images, test_labels_categorical, verbose=0)
    test_accuracies.append(test_accuracy)

# Calculate the average testing accuracy
average_test_accuracy = np.mean(test_accuracies)

# Print the results
print("Test Accuracies for each run:", test_accuracies)
print("Average Test Accuracy:", average_test_accuracy)

Epoch 1/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 61s 64ms/step - accuracy: 0.8017 - loss: 0.6066 - val_accuracy: 0.9832 - val_loss: 0.0522
Epoch 2/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 82s 64ms/step - accuracy: 0.9659 - loss: 0.1117 - val_accuracy: 0.9864 - val_loss: 0.0389
Epoch 3/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 82s 64ms/step - accuracy: 0.9755 - loss: 0.0803 - val_accuracy: 0.9892 - val_loss: 0.0325
Epoch 4/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 59s 63ms/step - accuracy: 0.9787 - loss: 0.0682 - val_accuracy: 0.9907 - val_loss: 0.0284
Epoch 5/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 81s 62ms/step - accuracy: 0.9822 - loss: 0.0594 - val_accuracy: 0.9905 - val_loss: 0.0288
Epoch 6/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 59s 63ms/step - accuracy: 0.9841 - loss: 0.0518 - val_accuracy: 0.9912 - val_loss: 0.0257
Epoch 7/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 83s 64ms/step - accuracy: 0.9840 - loss: 0.0508 - val_accuracy: 0.9917 - val_loss: 0.0232
Epoch 8/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 80s 63ms/step - accuracy: 0.9864 - loss: 0.0441 - 

The code is of Convolutional Neural Network which has been run over for 10 epochs and 5 times to get the average accuracy of the network.

The average test accuracy comes out to be: 99.22%.